In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import nibabel
from sklearn.model_selection import train_test_split

In [2]:
seg_path ='/content/drive/MyDrive/Colab Notebooks/MRI_Classification/Preprocessed_BraTS2021_TrainingData_16July2021/PreprocessedBraTS2021_00000_seg.nii.gz'

seg = nibabel.load(seg_path).get_fdata()
seg.shape


(160, 224, 192)

In [3]:
path = 'drive/MyDrive/Colab Notebooks/MRI_Classification/Preprocessed_BraTS2021_TrainingData_16July2021'

def padding(volume, xx, yy, zz):
    """
    pad a volume
    :param array: numpy array
    :param xx: desired height
    :param yy: desired width
    :param zz: desired depth
    :return: padded array
    """

    height = volume.shape[0]
    width = volume.shape[1]
    depth = volume.shape[2]

    a = (xx - height) // 2
    aa = xx - a - height

    b = (yy - width) // 2
    bb = yy - b - width
    
    c = (zz - depth) // 2
    cc = zz - c - depth
    
    return np.pad(volume, pad_width=((a, aa), (b, bb), (c,cc)), mode='constant')


In [ ]:

path = 'drive/MyDrive/Colab Notebooks/MRI_Classification/Preprocessed_BraTS2021_TrainingData_16July2021'
# go through the directorie and find all flair and segmentation files
#crop them according to the dimensions the flair files were cropped

count = 0
for root, dirs, files in os.walk(path):
    for name in files:
      count += 1
      if count%100 == 0:
        print(count)

      #load MRI data
      img_data = nibabel.load(os.path.join(root, name))
      img_array = img_data.get_fdata()
      img_affine = img_data.affine

      img_array = padding(img_array, 160, 224, 192)

      #for this project the image dimensions should be (160, 224, 192)
      assert(img_array.shape == (160, 224, 192))

      #convert to nibabel Nifti file
      nft_image = nibabel.Nifti1Image(img_array, img_affine)

      #save file
      nibabel.save(nft_image, os.path.join(path, name))




100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500


In [5]:
#make lists of the sample IDs

sample_list = []

for root, dirs, files in os.walk(path):
  for name in files:
    if "seg" in name:
      index = name.find('_')
      sample_key = name[index+1:index+6]
      flair_file = 'PreprocessedBraTS2021_' + sample_key + '_flair.nii.gz'
      if os.path.exists(os.path.join(path, flair_file)):
        sample_list.append(sample_key)

len(sample_list)

1251

In [7]:
# data generator for binary segmentation
# makes segmentation file elements 0 or 1

class TrainDataGenerator(keras.utils.Sequence):

    def __init__(self, sample_list, batch_size=1, dim=(160, 224, 192,), n_channels=1,
                n_classes=1, shuffle=True):
        """Constructor can be expanded,
           with batch size, dimentation etc.
        """
        self.sample_list = sample_list
        self.batch_size=batch_size
        self.dim=dim
        self.n_channels=n_channels
        self.n_classes=n_classes
        self.shuffle=shuffle
        self.on_epoch_end()

    def __len__(self):
        #Take all batches in each iteration'
        return int(np.floor(len(self.sample_list) / self.batch_size))

    def __getitem__(self, index):
        'Get next batch'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # single file
        sample_list_temp = [self.sample_list[k] for k in indexes]

        # Set of X_train and y_train
        X, y = self.__data_generation(sample_list_temp)

        return X, y

    def on_epoch_end(self):
        #Updates indexes after each epoch'
        self.indexes = np.arange(len(self.sample_list))
        #shuffle data is shuffle is true
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            

    def __data_generation(self, sample_list_temp):
        #Generates data containing batch_size samples'
        X = np.empty((self.batch_size, *self.dim), dtype=np.float32)
        y = np.empty((self.batch_size, *self.dim), dtype='uint8')
        
        # Generate data
        for i, sample in enumerate(sample_list_temp):
            x_path = 'PreprocessedBraTS2021_' + sample + '_flair.nii.gz'
            # Store sample
            X[i,] = nibabel.load(os.path.join(path, x_path)).get_fdata().astype('float32')

            y_path = 'PreprocessedBraTS2021_' + sample + '_seg.nii.gz'
            # Store class
            #new_y = (nibabel.load(os.path.join(path, y_path)).get_fdata() > 0).astype('uint8')
            #y[i,] = to_categorical(new_y, num_classes=2, dtype='uint8')
            y[i,] = (nibabel.load(os.path.join(path, y_path)).get_fdata() > 0).astype('uint8')
            
        return X, y


remain_samples, test_samples, _ , _ = train_test_split(sample_list, sample_list, test_size=0.1, random_state=42)

train_samples, val_samples, _ , _ = train_test_split(remain_samples, remain_samples, test_size=0.2222, random_state=42)

train_generator = TrainDataGenerator(train_samples)
val_generator = TrainDataGenerator(val_samples)

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, concatenate, BatchNormalization, Dense, Dropout, Flatten 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.layers import Activation, UpSampling3D, ReLU


def conv_block(input_layer, num_filters):
    x = Conv3D(filters=num_filters, kernel_size=5, strides=1, padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = ReLU(negative_slope=1.0)(x)
    
    x = Conv3D(filters=num_filters, kernel_size=5, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU(negative_slope=1.0)(x)
    
    x = Conv3D(filters=num_filters, kernel_size=5, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU(negative_slope=1.0)(x)
    
    return x

def encoder_block(input_layer, num_filters):
    x = conv_block(input_layer, num_filters)
    out = MaxPooling3D((2,2,2))(x)
    
    return out, x

def decoder_block(input_layer, conc_layer, num_filters):
    x = conv_block(input_layer, num_filters)
    x = UpSampling3D(size=2)(x)
    out = concatenate([conc_layer, x])
    
    return out
    
def build_unet(input_shape):

    input_layer = Input(input_shape)
    
    c1, u1 = encoder_block(input_layer, 8)
    c2, u2 = encoder_block(c1,16)
    c3, u3 = encoder_block(c2, 32)
    c4, u4 = encoder_block(c3, 64)
    
    c5 = decoder_block(c4, u4, 64)
    c6 = decoder_block(c5, u3, 32)
    c7 = decoder_block(c6, u2, 16)
    c8 = decoder_block(c7, u1, 8)
    
    segmentation_layer = Conv3D(filters=1, kernel_size=1, activation='sigmoid', padding='same')(c8)
    
    
    model = Model(input_layer, segmentation_layer, name='3D_semantic_segmentation')

    return model

#build model
input_shape = (160, 224, 192, 1)
model = build_unet(input_shape)
print(model.summary())

Model: "3D_semantic_segmentation"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 160, 224, 1  0           []                               
                                92, 1)]                                                           
                                                                                                  
 conv3d (Conv3D)                (None, 160, 224, 19  1008        ['input_1[0][0]']                
                                2, 8)                                                             
                                                                                                  
 batch_normalization (BatchNorm  (None, 160, 224, 19  32         ['conv3d[0][0]']                 
 alization)                     2, 8)                                      

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint('drive/MyDrive/Colab Notebooks/MRI_Classification/unet_model_3.h5', save_best_only=True)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)

callbacks = [checkpoint, early_stopping]

model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

epochs=10
history = model.fit(train_generator , 
                              validation_data=val_generator,
                             epochs=epochs,
                             verbose=1,
                             callbacks=callbacks
                   )


Epoch 1/10
437/437 [==============================] - 2992s 7s/step - loss: 0.2796 - accuracy: 0.9586 - val_loss: 0.3114 - val_accuracy: 0.8876
Epoch 2/10
437/437 [==============================] - 2076s 5s/step - loss: 0.0454 - accuracy: 0.9902 - val_loss: 0.1866 - val_accuracy: 0.9437
Epoch 3/10
437/437 [==============================] - 2074s 5s/step - loss: 0.0285 - accuracy: 0.9917 - val_loss: 0.0303 - val_accuracy: 0.9908
Epoch 4/10
437/437 [==============================] - 2072s 5s/step - loss: 0.0232 - accuracy: 0.9926 - val_loss: 0.0429 - val_accuracy: 0.9848
Epoch 5/10
437/437 [==============================] - 2072s 5s/step - loss: 0.0214 - accuracy: 0.9928 - val_loss: 0.0348 - val_accuracy: 0.9914
Epoch 6/10
437/437 [==============================] - 2073s 5s/step - loss: 0.0200 - accuracy: 0.9932 - val_loss: 0.0291 - val_accuracy: 0.9904
Epoch 7/10
437/437 [==============================] - 2072s 5s/step - loss: 0.0191 - accuracy: 0.9933 - val_loss: 0.0377 - val_accuracy:

In [48]:
def IoU_coef(ground_truth, y_preds, num_classes) -> float:
    '''
    Computes Intersection-Over-Union also known as Jaccard Index
    For multiclass task computes IoU individually for each class
        and then returns average. Class labels should begin with 0
        
    Inputs:
    ground_truth: a numpy array of the correct labels
    y_preds: a numpy array of the predicted labels
    num_classes: an int for the number of classes
    
    Output:
    [a float that is the average calculated IoU, a list of IoU for each class]
    '''
    
    iou_list = []
    
    for c in range(num_classes):
        ground_truth_labels = (ground_truth == c)
        prediction_labels = (y_preds == c)
        intersection = np.logical_and(ground_truth_labels, prediction_labels).sum()
        union =  ground_truth_labels.sum() + prediction_labels.sum() - intersection
        
        iou_list.append(intersection/union)
    
    
    
    return np.array(iou_list)

In [21]:
model.load_weights(filepath='drive/MyDrive/Colab Notebooks/MRI_Classification/unet_model_3.h5')

In [51]:
'''x = np.empty((1, 160,224,192), dtype=np.float32)
y = np.empty((1, 160,224,192), dtype=np.uint8)
IoU_totals = np.array([0,0])

for i in range(len(test_samples)):
  sample = test_samples[i]

  #load flair file
  file_path = os.path.join(path, 'PreprocessedBraTS2021_' + sample + '_flair.nii.gz')
  x = nibabel.load(file_path).get_fdata().astype('float32')

  #get model predictions
  y_pred = model.predict(x)
  y_pred = np.rint(y_pred)
  y_pred = np.reshape(y_pred, (1,160,224,192))

  #load ground truths
  file_path = os.path.join(path, 'PreprocessedBraTS2021_' + sample + '_seg.nii.gz')
  y = nibabel.load(file_path).get_fdata().astype('uint8')

  score = IoU_coef(y, y_pred, num_classes=2)
  IoU_totals = IoU_totals + score

IoU_totals = IoU_totals/len(test_samples)

print('IoU class 0:', IoU_totals[0])
print('IoU class 1:', IoU_totals[1])
print('IoU mean:', IoU_totals.mean())'''

"x = np.empty((1, 160,224,192), dtype=np.float32)\ny = np.empty((1, 160,224,192), dtype=np.uint8)\nIoU_totals = np.array([0,0])\n\nfor i in range(len(test_samples)):\n  sample = test_samples[i]\n\n  #load flair file\n  file_path = os.path.join(path, 'PreprocessedBraTS2021_' + sample + '_flair.nii.gz')\n  x = nibabel.load(file_path).get_fdata().astype('float32')\n\n  #get model predictions\n  y_pred = model.predict(x)\n  y_pred = np.rint(y_pred)\n  y_pred = np.reshape(y_pred, (1,160,224,192))\n\n  #load ground truths\n  file_path = os.path.join(path, 'PreprocessedBraTS2021_' + sample + '_seg.nii.gz')\n  y = nibabel.load(file_path).get_fdata().astype('uint8')\n\n  score = IoU_coef(y, y_pred, num_classes=2)\n  IoU_totals = IoU_totals + score\n\nIoU_totals = IoU_totals/len(test_samples)\n\nprint('IoU class 0:', IoU_totals[0])\nprint('IoU class 1:', IoU_totals[1])\nprint('IoU mean:', IoU_totals.mean())"

In [26]:
# data generator for test set
test_generator = TrainDataGenerator(test_samples)

#get predictions from model
probs = model.predict(test_generator)

In [36]:
probs.shape
probs_round = np.rint(probs).astype('uint8')

In [44]:
y_truth = np.empty((len(test_samples), 160,224,192,1), dtype=np.uint8)

In [50]:
for i in range(len(test_samples)):
  sample = test_samples[i]

  #load ground truths
  file_path = os.path.join(path, 'PreprocessedBraTS2021_' + sample + '_seg.nii.gz')
  y_truth[i,] = np.reshape(nibabel.load(file_path).get_fdata().astype('uint8'), (160,224,192,1))

IoU_totals = IoU_coef(probs_round, y_truth, num_classes=2)

print('IoU class 0:', IoU_totals[0])
print('IoU class 1:', IoU_totals[1])
print('IoU mean:', IoU_totals.mean())

IoU class 0: 0.9875676366115635
IoU class 1: 0.022296269629256505
IoU mean: 0.5049319531204101
